In [141]:
import pandas as pd
import numpy as np
import pulp 
from pulp import *
restaurants = pd.read_csv("restaurant_data.csv")

In [287]:
#User-define constants
price_expectation = 0.3
max_cost_per_meal = 55
max_transit_time_per_meal = 40
total_number_meal = 4
same_store_per_week = 2 

In [142]:
restaurants.drop(restaurants.index[(restaurants["Rating"]>5)],axis=0,inplace=True)
restaurants["Total No of Ratings"] = restaurants["Total No of Ratings"].map(lambda x: int(x.replace(",", "")))
restaurants.drop(restaurants.index[(restaurants["Total No of Ratings"]<20)],axis=0,inplace=True)
priceDict = {
  "Not Expensive": 2,
  "Average Cost": 3,
  "Expensive": 4,
  "Too Expensive": 5 
}

In [143]:
cost_list = restaurants["Cost"]
ind = 0
restaurants["price_exp"] = restaurants["Cost"]
for i in cost_list:
    restaurants.iloc[ind,18] = priceDict[i]
    ind = ind + 1
restaurants["Self_rating"] =  restaurants["Rating"]*(1-price_expectation) + restaurants["price_exp"]*price_expectation

In [144]:
restaurants.to_csv("rtest.csv")

In [288]:
#lp coefficients
r = list(restaurants["Self_rating"])
a = list(restaurants["Transit time"])
costDict = {
  "Not Expensive": 10,
  "Average Cost": 20,
  "Expensive": 40,
  "Too Expensive": 60 
}
b = list(restaurants["Cost"].map(lambda x: costDict[x]))
t = max_transit_time_per_meal*total_number_meal
c = max_cost_per_meal*total_number_meal
m = total_number_meal
n = same_store_per_week

prob2 = pulp.LpProblem("Restaurant_Problem", LpMaximize)
res_names = list(restaurants["Name of Restaurent "])
l = 0
for i in res_names:
    k = 0
    for j in res_names:
        if l != k:
            if i == j:
                res_names[k] = i+str(k)
        k = k+1
    l=l+1
rating = {res_names[i]:r[i] for i in range(0, 286)}
tr = {res_names[i]:a[i] for i in range(0, 286)}
co = {res_names[i]:b[i] for i in range(0, 286)}
rn_vars = LpVariable.dicts("rn", res_names, lowBound=0)
prob2 += sum(rating[i]*rn_vars[i] for i in res_names), "Total Points of the meals"

In [ ]:
print(prob2)

In [289]:
prob2 += lpSum([tr[i] * rn_vars[i] for i in res_names]) <= t, "timl"
prob2 += lpSum([co[i] * rn_vars[i] for i in res_names]) <= c, "cot"
prob2 += lpSum([rn_vars[i] for i in res_names]) <= m, "weekly_meal"



prob2.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob2.status])
for a in prob2.variables():
    print(a.name, "=", a.varValue)

Status: Optimal
rn_49th_Parallel_Caf?&_Lucky's_Doughnuts___MAIN = 0.0
rn_A&W_Canada = 0.0
rn_A&W_Canada242 = 0.0
rn_A&W_Canada243 = 0.0
rn_Absinthe_Bistro = 0.5
rn_Afghan_Horsemen_Restaurant = 0.0
rn_Aleph_Eatery = 0.0
rn_Altitudes_Bistro = 0.0
rn_Ancora_Waterfront_Dining_and_Patio___Ambleside = 0.0
rn_Ancora_Waterfront_Dining_and_Patio___False_Creek = 0.0
rn_Anh_and_Chi = 0.0
rn_AnnaLena = 0.0
rn_Ashiana_Tandoori = 0.0
rn_Ask_For_Luigi_Restaurant = 0.0
rn_Atlas_Steak___Fish = 0.0
rn_Au_Comptoir = 0.0
rn_Bacchus_Restaurant_&_Lounge = 0.0
rn_Baghdad_Cafe = 0.0
rn_Bao_Bei = 0.0
rn_Bauhaus_Restaurant = 0.0
rn_Bay_Sushi_Cafe_Express = 0.0
rn_Bella_Gelateria = 0.0
rn_Bellaggio_Cafe = 0.0
rn_Best_Neighbours_Restaurant_&_Pizza_House__Johnny?s_on_Oak = 0.0
rn_BiBo_Pizzeria_con_Cucina = 0.0
rn_Bibo_Pizzeria = 0.0
rn_Bishop's = 3.5
rn_Bistro_Verde = 0.0
rn_Black_Blue = 0.0
rn_Blue_Water_Cafe = 0.0
rn_Boulevard_Kitchen_&_Oyster_Bar = 0.0
rn_Bridges_Restaurant = 0.0
rn_Brix_&_Mortar = 0.0
rn_Burdo